In [22]:
import pandas as pd
import requests

In [54]:
!pip install plotly

     |████████████████████████████████| 7.1 MB 2.4 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=9f5c8c891511ff2ebd6a409ef028c488e74430c0f431dda2717623a4526e1766
  Stored in directory: /Users/Katalin/Library/Caches/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [56]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [73]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [18]:
df = pd.read_csv('data/RKI_Corona_Landkreise.csv')

In [5]:
df2 = pd.read_excel('data/COVID-19-geographic-disbtribution-worldwide-2020-03-20.xlsx')

### Aggreation

In [34]:
df3 = pd.read_csv('data/RKI_COVID19.csv')

In [44]:
df3.head()

,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,ObjectId,Meldedatum,IdLandkreis
0,15,Sachsen-Anhalt,SK Magdeburg,A35-A59,M,2,0,154936,2020-03-18T00:00:00.000Z,15003
1,15,Sachsen-Anhalt,SK Magdeburg,A35-A59,W,1,0,154937,2020-03-12T00:00:00.000Z,15003
2,15,Sachsen-Anhalt,SK Magdeburg,A35-A59,W,1,0,154938,2020-03-17T00:00:00.000Z,15003
3,15,Sachsen-Anhalt,SK Magdeburg,A60-A79,M,1,0,154939,2020-03-10T00:00:00.000Z,15003
4,15,Sachsen-Anhalt,SK Magdeburg,A60-A79,M,1,0,154940,2020-03-19T00:00:00.000Z,15003


In [47]:
df3[df3.Meldedatum=='2020-01-28T00:00:00.000Z']


,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,ObjectId,Meldedatum,IdLandkreis
4700,9,Bayern,LK Landsberg a.Lech,A15-A34,M,1,0,152404,2020-01-28T00:00:00.000Z,09181
4908,9,Bayern,LK Starnberg,A35-A59,M,1,0,152612,2020-01-28T00:00:00.000Z,09188


In [52]:
new_df = df3.groupby(['Meldedatum','Bundesland', 'Landkreis']).sum()[['AnzahlFall','AnzahlTodesfall']].reset_index()

In [95]:
new_df2 = df3.groupby(['Meldedatum','Bundesland']).sum()[['AnzahlFall','AnzahlTodesfall']].reset_index()

In [100]:
final_df = pd.DataFrame()
for landkreis in new_df.Landkreis.unique():
    sub_df = new_df[new_df['Landkreis'] ==landkreis].sort_values(by='Meldedatum')
    totals_ls = []
    totals_d_ls = []
    totals = 0
    totalsD = 0
    for i in range(0, len(sub_df)):
        totals = totals +sub_df.iloc[i].AnzahlFall
        totalsD = totalsD +sub_df.iloc[i].AnzahlTodesfall
        totals_ls.append(totals)
        totals_d_ls.append(totalsD)
    sub_df['TotalAnzahlLandkreis'] = totals_ls
    sub_df['TotalDeathLandkreis'] = totals_d_ls
    final_df = final_df.append(sub_df)

In [99]:
final_df2 = pd.DataFrame()
for landkreis in new_df2.Bundesland.unique():
    sub_df = new_df2[new_df2['Bundesland'] ==landkreis].sort_values(by='Meldedatum')
    totals_ls = []
    totals_d_ls = []
    totals = 0
    totalsD = 0
    for i in range(0, len(sub_df)):
        totals = totals +sub_df.iloc[i].AnzahlFall
        totalsD = totalsD +sub_df.iloc[i].AnzahlTodesfall
        totals_ls.append(totals)
        totals_d_ls.append(totalsD)
    sub_df['TotalAnzahlBundesland'] = totals_ls
    sub_df['TotalDeathBundesland'] = totals_d_ls
    final_df2 = final_df2.append(sub_df)

In [102]:
final_df2.to_csv('aggregate_bundesland.csv')

In [104]:
final_df.to_csv('aggregate_landkreisebende.csv')

In [92]:
fig = go.Figure()
timeline = new_df.Meldedatum
for landkreis in new_df.Landkreis.unique():


    fig.add_trace(go.Scatter(x=timeline, y= new_df[new_df[landkreis] ==landkreis].sort_values(by='Meldedatum').AnzahlFall,
                        mode='lines',
                        name=landkreis))

KeyError: 'LK Landsberg a.Lech'

In [ ]:
for i in range(0, len(new_df)):
    